!wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [1]:
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#mesolitica/malaysian-llama2-7b-32k-instructions-v2
tokenizer = AutoTokenizer.from_pretrained('mesolitica/mallam-5B-4096')
# model = AutoModelForCausalLM.from_pretrained(
#     'fpf-7b-instructions-16k-call-v2/checkpoint-9300', 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )

model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/mallam-5B-4096', 
    #use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="auto"
)

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:03<00:00,  1.09s/it]


In [3]:
from tqdm import tqdm
import os
import random
import json
random.seed(404)
torch.manual_seed(404)

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/BM-PT3/data/'
with open(data_folder + 'BM-A-pt3') as fopen:
    text = fopen.read()
print(text.split('no: '))

In [4]:
data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/fizik-spm/data/'
with open(data_folder+ 'fizik') as fopen:
    text = fopen.read()
    
    
questions = []
for t in text.split('no: ')[1:]: # split by "no"
    t = t.strip()
    no = t.split('\n')[0] # "split by newline to get index'
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [5]:
questions[0]

{'no': '1',
 'soalan': 'Antara yang berikut, yang manakah kuantiti vektor?\nA. Tenaga\nB. Jisim\nC. Daya\nD. Laju',
 'jawapan': 'C'}

In [6]:
arange = set(range(len(questions)))
print(arange)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [7]:
def convert_prompt(row, answer = False): # convert question to prompt 
    if answer: # include answer in prompt
        prompt = f""" 
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else: # do not include answer in prompt
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

def parse_mistral_chat(messages):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [11]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i])) # convert the question to prompt
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_mistral_chat(messages)
    print(prompt)
    break
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda') # get input ids 
    repeat = []
    for _ in range(5): # for each question repeat 5 times
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=50,
                top_p=0.95,
                top_k=50,
                temperature=0.9,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

  0%|                                                    | 0/20 [00:00<?, ?it/s]

<s>[INST] soalan: Antara yang berikut, yang manakah kuantiti vektor?
A. Tenaga
B. Jisim
C. Daya
D. Laju
jawapan: [/INST]


In [9]:
def most_common(l):
    return max(set(l), key=l.count)

In [10]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

0.0

In [13]:
for q in questions:
    print(q['output'], q['jawapan'])

['A'] C
['A'] B
[] D
['A', 'A'] C
['A', 'A'] B
['D', 'B'] D
['A', 'A', 'A'] C
['A'] D
['A'] B
['A', 'D'] B
['D', 'A'] D
['A'] B
[] D
['A', 'A'] B
['A'] C
[] A
['B'] D
['A'] C
['A'] C
['D'] A
